# Edited by Krish Naik 
    traget is having the highest score

In [ ]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('train.csv')
df.head()

In [ ]:
df['MSZoning'].value_counts()
df.isnull().sum() # view nb missing values by cloumns

In [ ]:
sns.heatmap(df.isnull(), yticklabels=False, cbar=False) # cmap='viridis'
df.shape; df.info() # see df RAM usage if is_large

In [ ]:
df['LotFrontage'] = df['LotFrontage'].fillna(df['LotFrontage'].mean())
df.drop(['Alley'], axis=1, inplace=True)

In [ ]:
df['BsmtCond']= df['BsmtCond'].fillna(df['BsmtCond'].mode()[0])
df['BsmtQual']= df['BsmtQual'].fillna(df['BsmtQuall'].mode()[0])

df['Fireplace'] = df['Fireplace'].fillna(df['Fireplace'].mode()[0])
df['GarageType'] = df['GarageType'].fillna(df['GarageType'].mode()[0])

df.drop(['GarageYrBlt'], axis=1, inplace=True)

df['GarageFinish'] = df['GarageFinish'].fillna(df['GarageFinish'].mode()[0])
df['GarageQual'] = df['GarageQual'].fillna(df['GarageQual'].mode()[0])
df['GarageCond'] = df['GarageCond'].fillna(df['GarageCond'].mode()[0])

df.drop(['PoolQC', 'Fence', 'MiscFeature'], axis=1, inplace=True)
df['MSZoning'].values_counts()

In [ ]:
df.drop(['Id'], axis=1, inplace=True)
df.isnull().sum()

In [ ]:
df['MasVnrType'] = df['MasVnrType'].fillna(df['MasVnrType'].mode()[0])
df['MasVnrArea'] = df['MasVnrArea'].fillna(df['MasVnrArea'].mode()[0])

# see null values by field on datasets
sns.heatmap(df.isnull(), yticklabels=False, cbar=False, cmap='coolwarm')
df['BsmtExposure'] = df['BsmtExposure'].fillna(df['BsmtExposure'].mode()[0])
df.dropna(inplace=True)
df.head()

In [ ]:
test_df = pd.read_csv('test.csv')
test_df.shape; test_df.head()

test_df.drop(['Alley'], axis=1, inplace=True)
test_df.shape

In [ ]:
columns = ['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'Neighboors', 'Condition2',
    'BldgType', 'Condition1', 'HouseStyle', 'SaleType', 'SaleCondition', 'ExterCond', 'ExterQual',
    'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']
len(columns)

In [ ]:
def catergory_onehot_multcols(multcolumns):
    df_final = final_df
    i = 0
    for fields in multcolumns:
        print(fields)
        df1 = pd.get_dummies(final_df[fields], drop_first=True)
        
        final_df = df.drop([fields], axis=1, inplace=True)
        if i == 0:
            df_final = df1.copy()
        else:
            df_final = pd.concat([df_final, df1], axis=1)
        i += 1
    df_final = pd.concat([final_df, df_final], axis = 1)
    return df_final

In [ ]:
main_df = df.copy()

In [ ]:
test_df = pd.read_csv('formulatedtest.csv')
test_df.head()
test_df.shape

In [ ]:

final_df = pd.concat([df, test_df], axis=0)

# remove duplicated columns
final_df = final_df.loc[:, ~final_df.columns.duplicated()]

final_df.shape
final_df = catergory_onehot_multcols(columns)
final_df.head()

In [ ]:
df_Train = final_df.iloc[:1422, :]
df_Train = final_df.iloc[1422: , :]

In [ ]:
df_Test = None
df_Test.drop(['SalePrice'], axis=1, inplace=True)
df_Train['SalePrice'].hist()

In [ ]:
df_Train.drop(['Wall'], axis=1, inplace=True)
df_Train.corr()['SalePrice']

In [ ]:
X_train = df_Train.drop(['SalePrice'], axis=1)
y_train = df_Train['SalePrice']

In [ ]:
import xgboost
from sklearn.ensemble import RandomForestRegressor

classifier = xgboost.XGBRegressor() # not hyperized yet
classifier.fit(X_train, y_train)

In [ ]:
regressor = xgboost.XGBRegressor() # compare with RandomForestRegressor() next

# Hyper Parameter Optimization

In [ ]:
n_estimators = [100, 500, 900, 1100, 1500]
max_depth = [2, 3, 5 ,10, 15]
booster = ['gbtree', 'gblinear']
learning_rate = [0.05, 0.1, 0.15, 0.20]
min_child_wieght = [1, 2, 3, 4]
base_score = [0.25, 0.5, 0.75, 1]

# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth': max_depth,
    'booster': booster,
    'learning_rate': learning_rate,
    'min_child_wieght': min_child_wieght, 
    'base_score': base_score
}

#### *Set up Random search with 4-fold cross validation*

In [ ]:
# import RandomizedSearchCV

random_cv = RandomizedSearchCV(estimator=regressor, param_distributions=hyperparameter_grid, random_state = 42,
    cv = 5, n_iter = 50, scoring='neg_mean_absolute_error', n_jobs=4, verbose = 5, return_train_score = True)

random_cv.fit(X_train, y_train)

In [ ]:
regressor = random_cv.best_estimator_ # shows the param_values required
"""hyperparameter-optimization of the model {regressor}
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1, colsample_bytree=1, silent=True, max_depth=3,
    learning_rate=0.05, min_child_wieght=1, missing=None, n_estimators=900, n_jobs=1, nthread=None, seed=None, 
    objective='reg:linear', random_state=0, reg_aplha=0, reg_lambda=2, scale_pos_weight=1, gama=0, subsample=1)
"""
regressor.fit(X_train, y_train)

In [ ]:
import pickle

filename = 'finalized_model.pkl'
pickle.dump(classifier, open(filename, 'wb'))
y_pred = regressor.predict(df_Test)
print(y_pred)

In [ ]:
rf_pred = regressor.predict(df_Test)

## Create sample submission file and Submit

In [ ]:
pred = pd.DataFrame(y_pred)
sub_df = pd.read_csv('sample_submission.csv')
datasets = pd.concat([sub_df['Id'], pred], axis=1)
datasets.columns = ['Id', 'SalePrice']
datasets.to_csv('sample_submission.csv', index=False)

In [ ]:
rf_pred = pd.DataFrame(rf_pred) # This is a random_forest
sub_df = pd.read_csv('sample_submission.csv')
datasets = pd.concat([sub_df['Id'], rf_pred], axis=1)
datasets.columns = ['Id', 'SalePrice']
datasets.to_csv('sample_submission.csv', index=False)

# Typos
    use correlation
    model.fit(df_Train + df_Test) 
    remove non-relevant data_fields out_subject of the thème

In [ ]:
corr = df_Train.corr()
corr.head()

In [ ]:
columns = np.full((corr.shape[0], ), True, dtype=bool)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i, j] >= 0.9:
            if columns[j]: columns[j] = False

selected_columns = df_Train.columns[columns]
data = df_Train[selected_columns]
print(columns)